CPace sage notebook for generating test vectors for the CPace25519 cipher suite.

The notebook is organized in a series of several subcells.
1.) X25519 definitions and basic string->integer->fieldElement->string conversions back and forth.
2.) Definitions of the Elligator2 primitive (both, one derived from the Hash2Curve code and 
    one with straight-line code from the Elligator paper.
3.) Test vector generation for X25519 and Elligator2 as ported to sage from the ANSI-C code from the
    Endress+Hauser crypto library.
4.) Implementation and test vector generation for the CPace-specific constructions for generator
    calculation and ISK determination.
    
The cells should best be calculated in sequence so that all definitions are available.
Tested with SageMath 8.6 on ubuntu linux.

In [3]:
# 1.) X25519 definitions and tool definitions
   
def ByteArrayToInteger(k,numBytes=32):
    return sum((k[i] << (8 * i)) for i in range(len(k)))

def IntegerToByteArray(k,numBytes = 32):
    result = bytearray(numBytes);
    for i in range(numBytes):
        result[i] = (k >> (8 * i)) & 0xff;
    return result

def IntegerToLEPrintString(u,numBytes=32):
    u = Integer(u)
    res = ""
    ctr = 0
    while ((u != 0) | (numBytes > 0)):
        byte =  u % 256
        res += ("%02x" % byte)
        u = (u - byte) >> 8
        numBytes = numBytes - 1
        ctr = ctr + 1
        if (ctr % 32) == 0:
            res += "\n"
    return res


def ByteArrayToCInitializer(k, name):
    values = [b for b in k]
    result = "const uint8_t " + name +"[] = {\n "
    for x in values:
        result += hex(x) +","
    result += "\n};"
    return result

def ByteArrayToLEPrintString(k):
    bytes = [(b) for b in k]
    res = ""
    ctr = 0
    for x in bytes:
        res += ("%02x" %x)
        ctr = ctr + 1
        if (ctr % 32) == 0:
            res += "\n"        
    return res

########## X25519 ##################

def clampScalar25519(k):    
    k[0] &= 248
    k[31] &= 127
    k[31] |= 64
    return k

def clampScalarForInversion25519(k):
    k[0] &= 248
    return k

# all inputs to be given as byte array.
def Inverse_X25519(basepoint,scalar):
    OrderPrimeSubgroup = 2^252 + 27742317777372353535851937790883648493
    SF = GF(OrderPrimeSubgroup)
    coFactor = SF(8)
    scalarClamped = clampScalar25519(scalar)
    inverse_scalar = 1 /  (SF(ByteArrayToInteger(scalarClamped)) * coFactor)
    inverse_scalar_int = Integer(inverse_scalar) * 8
    inverse_scalar = IntegerToByteArray(inverse_scalar_int)
    return X25519(basepoint,inverse_scalar,withClamping=0)

def X25519(basepoint, scalar, withClamping=1):
    prime = 2^255 - 19
    F = GF(prime)
    A = F(486662)
    nonsquare = F(2)   
    E = EllipticCurve(F, [0, A , 0, 1 , 0])
    Twist = EllipticCurve(F, [0, A * nonsquare, 0, 1 * nonsquare^2, 0])

    u = F(ByteArrayToInteger(basepoint))
    if (withClamping == 1):
        clampedScalar = ByteArrayToInteger(clampScalar25519(scalar))
    else:    
        clampedScalar = ByteArrayToInteger(clampScalarForInversion25519(scalar))

        
    d = 1
    v2 = u^3 + A*u^2 + u
    if not v2.is_square():
        print("Input point is on the twist! "),
        E = Twist
        d = nonsquare
        u = d * u
        v2 = u^3 + A*u^2 * nonsquare + u * nonsquare^2
    v = v2.sqrt()
    
    point = E(u, v)
    (resultPoint_u, resultPoint_v, result_Point_z) = point * clampedScalar
    resultCoordinate = resultPoint_u / d
    
    return IntegerToByteArray(Integer(resultCoordinate))


In [16]:
# 2.) Definitions for the X25519 test cases

class X25519_testCase:
    def __init__(self,u_in, s_in, u_out):
        self.u_in = u_in
        self.s_in = s_in
        self.u_out = u_out

    def runTest(self):
        us = IntegerToByteArray(self.u_in)
        ss = IntegerToByteArray(self.s_in)
        r  = IntegerToByteArray(self.u_out)
        u = X25519(us,ss)
        if (u != r):
            print ("Fail")
            print ("Input u :\n0x%032x\n" % self.u_in)
            print ("Input s :\n0x%032x\n" % self.s_in)
            print ("Correct Result :\n0x%032x\n" % self.u_out)
            print ("Actual Result :\n0x%032x\n" % ByteArrayToInteger(u))
            return False
        print ("Pass")
        return True
    
    def docOutput(self):
        print ("Test case for X25519:")
        print ("u:"),
        print (IntegerToLEPrintString(self.u_in))
        print ("s:"),
        print (IntegerToLEPrintString(self.s_in))
        print ("r:"),
        print (IntegerToLEPrintString(self.u_out))
        

testCases = []

tv = \
    X25519_testCase(0x4c1cabd0a603a9103b35b326ec2466727c5fb124a4c19435db3030586768dbe6,\
                    0xc49a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346a5,\
                    0x5285a2775507b454f7711c4903cfec324f088df24dea948e90c6e99d3755dac3)
testCases.append(tv)


tv = X25519_testCase(0x13a415c749d54cfc3e3cc06f10e7db312cae38059d95b7f4d3116878120f21e5,\
                     0xdba18799e16a42cd401eae021641bc1f56a7d959126d25a3c67b4d1d4e9664b,\
                    0x5779ac7a64f7f8e652a19f79685a598bf873b8b45ce4ad7a7d90e87694decb95)
testCases.append(tv)

tv = X25519_testCase(0,\
                     0xc49a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346a5,\
                     0)
testCases.append(tv)
    
weakp = []
weakp.append(0)
weakp.append(1)
weakp.append(325606250916557431795983626356110631294008115727848805560023387167927233504) #(which has order 8)
weakp.append(39382357235489614581723060781553021112529911719440698176882885853963445705823) #(which also has order 8)
weakp.append(2^255 - 19 - 1)
weakp.append(2^255 - 19)
weakp.append(2^255 - 19 + 1)
weakp.append(2^255 - 19 + 325606250916557431795983626356110631294008115727848805560023387167927233504)
weakp.append(2^255 - 19 + 39382357235489614581723060781553021112529911719440698176882885853963445705823)
weakp.append(2 * (2^255 - 19) - 1)
weakp.append(2 * (2^255 - 19))
weakp.append(2 * (2^255 - 19) + 1)

for x in weakp:
    tv = X25519_testCase (x,0xff9a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346af,0)
    testCases.append(tv)

for x in testCases:
    x.runTest()

for x in testCases:
    x.docOutput()


Pass
Input point is on the twist! 
Pass
Pass
Pass
Pass
Pass
Pass
Input point is on the twist! 
Pass
Pass
Pass
Pass
Pass
Input point is on the twist! 
Pass
Pass
Pass
Test case for X25519:
u:
e6db6867583030db3594c1a424b15f7c726624ec26b3353b10a903a6d0ab1c4c

s:
a546e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a2244ba449ac4

r:
c3da55379de9c6908e94ea4df28d084f32eccf03491c71f754b4075577a28552

Test case for X25519:
u:
e5210f12786811d3f4b7959d0538ae2c31dbe7106fc03c3efc4cd549c715a413

s:
4b66e9d4d1b4673c5ad22691957d6af5c11b6421e0ea01d42ca4169e7918ba0d

r:
95cbde9476e8907d7aade45cb4b873f88b595a68799fa152e6f8f7647aac7957

Test case for X25519:
u:
0000000000000000000000000000000000000000000000000000000000000000

s:
a546e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a2244ba449ac4

r:
0000000000000000000000000000000000000000000000000000000000000000

Test case for X25519:
u:
0000000000000000000000000000000000000000000000000000000000000000

s:
af46e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a22

In [17]:
#3.) Function definitions for Elligator2
#
# Elligator2 
#
# Naive implementation from the Elligator paper

def elligator2_curve25519(r):
    p = 2^255 - 19
    F = GF(p)
    A = F(486662)
    B = F(1)
    r = F(ByteArrayToInteger(r,len(r)))
    
    u = F(2) # Our Non-square
    powerForChi = 2^254 - 10
    
    v = - A / (1 + u * r^2)
    epsilon = (v^3 + A * v^2 + B * v)^powerForChi
    x = epsilon * v - (1 - epsilon) * A/2
    return IntegerToByteArray(Integer(x))


In [7]:
# 4.) Definitions of the Elligator2 test cases

    
#
# Elligator 2 test cases
#
#
# Testvector from the NaCl M0 testsuite from E+H
#

etc1_in =  0x00c84eddfa9bcd7973d6021153cd965a8a2fd749135834eaaeb093d2469a14bc
etc1_out = 0x67d305efdb0a7c7f24ce1655ecc103126004ff23d32bfc032428cd75758fb666

etc2_in =  0x7563f23b0c0aa7bc27b2961a4711ba842ba303c57a9534164bf8d3b5d455cf89
etc2_out = 0x08a3bb40e5b594b192d0ee87b663d24e1bc76d2d41c9031962a7ec6cc863b11d

ourResult1 = ByteArrayToInteger(elligator2_curve25519(IntegerToByteArray(etc1_in)))
ourResult2 = ByteArrayToInteger(elligator2_curve25519(IntegerToByteArray(etc2_in)))

if (ourResult1 != etc1_out):
    print ("Elligator test case #1 failed.")
else:
    print ("Elligator test case #1 pass.")

if (ourResult2 != etc2_out):
    print ("Elligator test case #2 failed.")
else:
    print ("Elligator test case #2 pass.")
    
print ("Elligator test case #1:")
print ("In:  0x%x" % etc1_in)
print ("Out: 0x%x" % etc1_out)

print ("Elligator test case #1:")
print ("In:  0x%x" % etc2_in)
print ("Out: 0x%x" % etc2_out)


Elligator test case #1 pass.
Elligator test case #2 pass.
Elligator test case #1:
In:  0xc84eddfa9bcd7973d6021153cd965a8a2fd749135834eaaeb093d2469a14bc
Out: 0x67d305efdb0a7c7f24ce1655ecc103126004ff23d32bfc032428cd75758fb666
Elligator test case #1:
In:  0x7563f23b0c0aa7bc27b2961a4711ba842ba303c57a9534164bf8d3b5d455cf89
Out: 0x8a3bb40e5b594b192d0ee87b663d24e1bc76d2d41c9031962a7ec6cc863b11d


In [27]:
# 4.) Definitions for the CPace25519 specific primitives

# We need SHA-512
import hashlib 

def map_to_group_mod_neg_CPace25519(sid, PRS, CI, doPrint = 1, DSI1 = b"CPace25519-1"):
    F = GF(2^255 - 19)
    m = hashlib.sha512()
    H_block_SHA512 = 128
    ZPAD_len = max(0,H_block_SHA512 - len(DSI1) - len(PRS))
    ZPAD = ZPAD_len * b"\0"
        
    if doPrint:
        print ("DSI11= %s string (%s) of len(%i)" % (ByteArrayToLEPrintString(DSI1),DSI1, len(DSI1)))
        print ("PRS  = %s (%s) string of len(%i)" % (ByteArrayToLEPrintString(PRS), PRS, len(PRS)))
        print ("ZPAD = %i zero bytes (before mixing in adversary controlled variable data)" % (len(ZPAD)))
        print ("sid  = %s string of len(%i)" % (ByteArrayToLEPrintString(sid), len(sid)))
        print ("CI   = %s\n       (%s) string of len(%i)\n" % (ByteArrayToLEPrintString(CI), CI,len(CI)))
    m.update(DSI1)
    m.update(PRS)
    m.update(ZPAD)
    m.update(sid) 
    m.update(CI)
    digest = m.digest();
    u = ByteArrayToInteger(digest,64)
    if doPrint:
        print ("u = SHA512(DSI1||PRS||ZPAD||sid||CI) as 512 bit little-endian int:\n  0x%032x << 256\n+ 0x%32x" \
               % (Integer(u) >> 256, Integer(u) & (2^256 - 1)))
    u = F(u)
    print ("u as reduced base field element coordinate:\n  0x%x" % Integer(u))
    point = elligator2_curve25519(digest)
    print ("Elligator2 output G as base field element coordinate:\n  0x%x" % ByteArrayToInteger(point))
    return point

def KDF_CPace25519(sid,K,Ya,Yb,doPrint = 1):
    DSI2 = b"CPace25519-2"
    m = hashlib.sha512(DSI2)
    m.update(sid) 
    m.update(K)
    m.update(Ya)
    m.update(Yb)
    ISK = m.digest()
    if doPrint:
        print ("DSI2 = %s string (%s) of len(%i)" % (ByteArrayToLEPrintString(DSI2),DSI2, len(DSI2)))
        print ("sid  = %s string of len(%i)" % (ByteArrayToLEPrintString(sid), len(sid)))
        print ("K    = %s"\
               "       string of len(%i)" % (ByteArrayToLEPrintString(K), len(K)))
        print ("Ya   = %s"\
               "       string of len(%i)" % (ByteArrayToLEPrintString(Ya), len(Ya)))
        print ("Yb   = %s"\
               "       string of len(%i)" % (ByteArrayToLEPrintString(Yb), len(Yb)))
        print ("ISK  = SHA512(DSI2 || sid || K || Ya || Yb)\n" \
               "     = %s" \
               "       string of len(%i)" % (ByteArrayToLEPrintString(ISK), len(ISK)))
    return ISK


sid = hashlib.sha512(b"sid").digest()
sid = sid [:16]
PRS = b"password"
CI = b"AinitiatorBresponderAD"

print ("Derivation of the secret generator G\n")
G = map_to_group_mod_neg_CPace25519(sid,PRS,CI,1)

ya = hashlib.sha512(b"ya").digest()
ya = bytearray(ya [:32])
yb = hashlib.sha512(b"yb").digest()
yb = bytearray(yb [:32])

Ya = X25519(G,ya)
Yb = X25519(G,yb)

K1 = X25519(Yb,ya)
K2 = X25519(Ya,yb)

if (K1 != K2):
    print ("Diffie-Hellman did fail!")
else:
    print ("Diffie-Hellman pass.")
    K = K1
    
print ("\nSecret scalar ya=SHA512(\'ya\'), bytes 0...31, as integer:\n  0x%x" % ByteArrayToInteger(ya))
print ("Secret scalar yb=SHA512(\'yb\'), bytes 0...31, as integer:\n  0x%x" % ByteArrayToInteger(yb))

print ("Public point Ya as integer:\n  0x%x" % ByteArrayToInteger(Ya))
print ("Public point Yb as integer:\n  0x%x" % ByteArrayToInteger(Yb))

print ("DH point K as integer:\n  0x%x" % ByteArrayToInteger(K1))

print ("\nDerivation of the intermediate session key ISK:\n")
ISK = KDF_CPace25519(sid,K,Ya,Yb,1)

Derivation of the secret generator G

DSI11= 435061636532353531392d31 string (b'CPace25519-1') of len(12)
PRS  = 70617373776f7264 (b'password') string of len(8)
ZPAD = 108 zero bytes (before mixing in adversary controlled variable data)
sid  = 7e4b4791d6a8ef019b936c79fb7f2c57 string of len(16)
CI   = 41696e69746961746f7242726573706f6e6465724144
       (b'AinitiatorBresponderAD') string of len(22)

u = SHA512(DSI1||PRS||ZPAD||sid||CI) as 512 bit little-endian int:
  0xd00895f55108c0746da4aa584aa611714037149055a981076edac21a582be61a << 256
+ 0x4409b1557780fc3738b8d7e51fe459659e5eead08948af5665a52beb84eac23f
u as reduced base field element coordinate:
  0x254ff3bf7ecd8d7f7f2a2100348af035268bf83d4071d670da1dfbd49b6eeeb5
Elligator2 output G as base field element coordinate:
  0x41bfa27149d614f163d64dcb5b47da8991105a41065a5447fda1b7cf76e00ec4
Diffie-Hellman pass.

Secret scalar ya=SHA512('ya'), bytes 0...31, as integer:
  0x7fec93334144994275a3eba9eb0adf3fe40d54e400d105d59724bee398b722d0
Sec

In [9]:
# Test the inverse X25519 operation for strong AuCPace.

# generate deterministic random numbers based on SHA512 and different strings.
# Generate points and scalars from such random inputs.
# Check that the inverse operation is actually inverting X25519.

inStr = "a"
for m in range(100):
    B = IntegerToByteArray(9)

    s = hashlib.sha512(inStr.encode()).digest()
    inStr += "x"
    s = bytearray(s [:32])

    T = X25519(B,s)
    print ("T : 0x%x" % ByteArrayToInteger(T))

    inStr += "x"

    r = hashlib.sha512(inStr.encode()).digest()
    r = bytearray(r [:32])
    print ("r : 0x%x" % ByteArrayToInteger(r))

    U = X25519(T,r)
    print ("U : 0x%x" % ByteArrayToInteger(U))

    IU = Inverse_X25519(U,r)
    #integerToLittleEndianString(Inverse_X25519(U,r))
    print ("IU : 0x%x" % ByteArrayToInteger(IU))
    error = ByteArrayToInteger(IU) - ByteArrayToInteger(T)
    if (error):
        print ("Error : 0x%x" % (error))

print ("End")

T : 0x5d7189be6192ffccdb80902ac26c5d38592822a761c7268007200a232a4cd841
r : 0x943f6e29367840ecf9035dfff06cbcda53c06ea13c6925ae17de6d1016b62bc0
U : 0x489b613d7d7068238cc72d5fd66d52adae82057526b67ff0332bc85f158fb7f8
IU : 0x5d7189be6192ffccdb80902ac26c5d38592822a761c7268007200a232a4cd841
T : 0x6a1c902cfa176af32437d097b26938e8c6e6129814dc6d0894da60f6e8a1c769
r : 0xf45d4141eff12fd4e6c3d30ec89d250987cd848a44833fbe173898824e82a5b7
U : 0x7d225c0ebd03860596e21fb7a5e49d0da22ef9e327fc342c3f56f7f2391efeab
IU : 0x6a1c902cfa176af32437d097b26938e8c6e6129814dc6d0894da60f6e8a1c769
T : 0x5c7d2a764389be0b53acaeb308fbd0828947fb4d30fa8778c27ac6a92d4d96fb
r : 0x1ae5f75f8157a437f8a66464a7176ddbc35d7a9190b45bf71c22c71e8841f87e
U : 0x3798d6267f45a734e458d8819d8f30ff21a2f1e939b9c605fea0a4ba10e8d2b5
IU : 0x5c7d2a764389be0b53acaeb308fbd0828947fb4d30fa8778c27ac6a92d4d96fb
T : 0x799c700bb7515cefd7beb6f53cb425bdd32838c306e3a4ec52119ea9ffb27523
r : 0x7616b776ce399e3dad59d61add5d1c4c506e271c3ca8f9a08f10ec82ac971a1d
U :

T : 0x77969851e4cd29f76fd3d2963f421b0c584708dafb2c991cf502a3c8e4dae570
r : 0x228bc68fbf20cba800374410ed3d293cdb5b5219cbdb2f20bd4a93ad58bdb92a
U : 0x2f6beeb60338bbc937b9ddece873a9d3c2dc5757b34d931fa92dd58f8d6dc7c5
IU : 0x77969851e4cd29f76fd3d2963f421b0c584708dafb2c991cf502a3c8e4dae570
T : 0x49a14f57eb9a83c2c6fba366077284adf5e811b67c7b1c151a842f5e213c90ca
r : 0xd769f3762dfe87e9ff27f5c3d480b49d5f15ce476a042665230342d8ae8d5568
U : 0x3c512b70f3a8d7f1c12b38acc4d337dad8a2d2ca292a3b3b3dafbd229a2199cb
IU : 0x49a14f57eb9a83c2c6fba366077284adf5e811b67c7b1c151a842f5e213c90ca
T : 0x49829050854de81dcb5f1ba76531d19c549b2403b9cb2de714b60d0f0d02752e
r : 0x66ead82728786cf2badd46a901b5722995c04fca7b43b802f28f15e51833fbf2
U : 0x2c38dd6faa7f1d9645c79f96350f5c897ff11bc347ab3f8ad06bf64290f82d7b
IU : 0x49829050854de81dcb5f1ba76531d19c549b2403b9cb2de714b60d0f0d02752e
T : 0x7a9f7ff6dc0850e45572a9e1d01be1795d3374bf74370d0e70622b80f9d17e6c
r : 0x247f29897c3bb8074d37f2877f1f331b2993153639c3e833e8ffaf95de7e9180
U :

T : 0x62f604047c9a53f3b3473ccd58832350a06622a8c003d2d69d035bfc40f3ab55
r : 0x8860306e46042f91ba6be025fb90583878bffb0ac612bac4d2550f3a1737043a
U : 0x66af6f941c24ff999c777959b4f8b422942a6a0b37f9caedf893c5283cdca5ec
IU : 0x62f604047c9a53f3b3473ccd58832350a06622a8c003d2d69d035bfc40f3ab55
T : 0x7c763f231fdba8e1913e416486f291d040d309dfc719ba6c3d1fdc5ad8b0e326
r : 0x4f05b5d5ca7f45fa7b36fe921338cbb4be6d273febc4aa396bcf845e67c5cc58
U : 0x5cd84402bcd9dcadb68e9217b2eb456e75f94969cbddceb4b8e61c9d95c4a813
IU : 0x7c763f231fdba8e1913e416486f291d040d309dfc719ba6c3d1fdc5ad8b0e326
T : 0x4490b8e2a3535ab63c7eeba2ec4a64e8fe9724233fccfcf631e5bfeb7ec1ace7
r : 0x63d579d972d444db817b88f5819dd83ea23be1e2b2df37f2a284324082157ad2
U : 0x6c218aa6c86e103576df7e760324784efd85f516345753a210277e3e192debc4
IU : 0x4490b8e2a3535ab63c7eeba2ec4a64e8fe9724233fccfcf631e5bfeb7ec1ace7
T : 0x24bf49ef5b3fa4d682eb9360756abe70ef82d3e6ac53ea1d8029551f6afcd624
r : 0x35d8fc648cca40f527850bfdc4f3dd63b28117fa966cc7272de4b704a1e0fcc4
U :

U : 0x4f4a608e583c80f6381d3b1c2600412b59bdca25a80d230a778edf65889feccc
IU : 0x1230cf1e6dbe9b9fb6116f681c983b11a9fcf75d04c0d5187915a64ea665ff8f
T : 0x31a45fc60ffd71b7dd4d8cd4f6cd5cbdfbc612ad6e0c4f7615c6cf536228bd5a
r : 0xfec5d3690ff27a4d9218e30fb98d600e42abd5808530e248b675a2d1676c3368
U : 0x11e78bb1ffd5e96c2240bbab842f6530570bc94810e2a2fbb0c423b33a16d757
IU : 0x31a45fc60ffd71b7dd4d8cd4f6cd5cbdfbc612ad6e0c4f7615c6cf536228bd5a
T : 0x7e86574fc4e4621de99a8ce32244213255e17be1a54e722a134863f21423ed9c
r : 0x8155956e53ab88d1b53ff2568890de42f6d47b4ad534f72deac223162a2e46b5
U : 0x7fee6ba5e3ce23c518b2f0ba6bec322836f022e8898282828ef831cea02f7d4e
IU : 0x7e86574fc4e4621de99a8ce32244213255e17be1a54e722a134863f21423ed9c
T : 0x8e56acf397b63d8edd99a6be45dab96efa1116485e9d4fb2f226109d4457bd7
r : 0x7e823424e00f5090883268d8d12ef974eab038abd596a0562ccbf2bd5db3d168
U : 0x6d6248820bc3974f943508391354a3683ef777feb7208be336d6187475761fc2
IU : 0x8e56acf397b63d8edd99a6be45dab96efa1116485e9d4fb2f226109d4457bd7
T : 

In [26]:
#Definition of the primitives for AuCPace

def map_to_group_mod_neg_AuCPace25519(username, password, doPrint = 1, DSI = b"AuCPace25519"):
    F = GF(2^255 - 19)
    m = hashlib.sha512()
    H_block_SHA512 = 128
    ZPAD_len = max(0,H_block_SHA512 - len(DSI) - len(password))
    ZPAD = ZPAD_len * b"\0"
        
    if doPrint:
        print ("DSI5 = %s string (%s) of len(%i)" % (ByteArrayToLEPrintString(DSI),DSI, len(DSI)))
        print ("pw   = %s (%s) string of len(%i)" % (ByteArrayToLEPrintString(password), password, len(password)))
        print ("ZPAD = %i zero bytes (before mixing in adversary controlled variable data)" % (len(ZPAD)))
        print ("name = %s (%s) string of len(%i)" % (ByteArrayToLEPrintString(username), username, len(username)))
    m.update(DSI)
    m.update(password)
    m.update(ZPAD)
    m.update(username) 
    u = ByteArrayToInteger(bytearray(m.digest()),64)
    
    if doPrint:
        print ("u = SHA512(DSI||password||ZPAD||username) as 512 bit little-endian int:\n  0x%032x << 256\n+ 0x%32x" \
               % (Integer(u) >> 256, Integer(u) & (2^256 - 1)))
    u = F(u)
    print ("u as reduced base field element coordinate:\n  0x%x" % Integer(u))
    Z = elligator2_curve25519(IntegerToByteArray(Integer(u)))
    print ("Elligator2 output Z as base field element coordinate:\n  0x%x" % ByteArrayToInteger(Z))
    return Z

def MAC_SK_AuCPace25519(ISK,doPrint = 1):
    DSI3 = b"AuCPace25-Ta"
    DSI4 = b"AuCPace25-Tb"
    DSI5 = b"AuCPace25519"
    m = hashlib.sha512(DSI3)
    m.update(ISK) 
    Ta = m.digest()
    Ta = Ta[:16]

    m = hashlib.sha512(DSI4)
    m.update(ISK) 
    Tb = m.digest()
    Tb = Tb[:16]

    m = hashlib.sha512(DSI5)
    m.update(ISK) 
    SK = m.digest()
    if doPrint:
        print ("DSI3 = %s string (%s) of len(%i)" % (ByteArrayToLEPrintString(DSI3),DSI3, len(DSI3)))
        print ("DSI4 = %s string (%s) of len(%i)" % (ByteArrayToLEPrintString(DSI4),DSI4, len(DSI4)))
        print ("DSI5 = %s string (%s) of len(%i)" % (ByteArrayToLEPrintString(DSI5),DSI5, len(DSI5)))
        print ("ISK  = %s string of len(%i)" % (ByteArrayToLEPrintString(ISK), len(ISK)))
        print ("Ta  = %s string of len(%i)" % (ByteArrayToLEPrintString(Ta), len(Ta)))
        print ("Tb  = %s string of len(%i)" % (ByteArrayToLEPrintString(Tb), len(Tb)))
        print ("SK  = %s string of len(%i)" % (ByteArrayToLEPrintString(SK), len(SK)))
    return (Ta,Tb,SK)

name = b"username"
password = b"password"

q = hashlib.sha512(b"q").digest()
q = bytearray(q [:32])

r = hashlib.sha512(b"r").digest()
r = bytearray(r [:32])

Z = map_to_group_mod_neg_AuCPace25519(name,password)

print ("\n\n\n")
print ("Z for username, password:\n0x%x" % ByteArrayToInteger(Z))

print ("q:\n0x%x" % ByteArrayToInteger(q))


U = X25519(Z,r)
print ("R:\n0x%x" % ByteArrayToInteger(r))

print ("U:\n0x%x" % ByteArrayToInteger(U))

UQ = X25519(U,q)

print ("UQ:\n0x%x" % ByteArrayToInteger(UQ))

ZQ = X25519(Z,q)

print ("ZQ directly calculated:\n0x%x" % ByteArrayToInteger(ZQ))

ZQ2 = Inverse_X25519(UQ,r)
print ("ZQ as calculated from UQ:\n0x%x" % ByteArrayToInteger(ZQ2))



def ByteArrayToString(k):
    chars = [chr(b) for b in k]
    result = ""
    for x in chars:
        result += x
    return result

import scrypt

w = scrypt.hash(password + name, ByteArrayToString(ZQ), N=1 << 15, r=8, p=1)
w = bytearray(w[:32])
W = X25519(B,w)

print("Password: \'%s\', length %i" % (password,len(password)))
print("User Name: \'%s\', length %i" % (name,len(name)))
print("Concatenated input to scrypt: %s \'%s\', length %i" % (ByteArrayToLEPrintString(password+name),\
                                                              (password+name), \
                                                             len(password+name)))
print("Salt value Z^q:\n%s" % ZQ)
print("Password hash w from scrypt with N=1<<15, r=8, p=1\n%s" % ByteArrayToLEPrintString(w))

print("Password verifier W = X25519(w)\n0x%x" % ByteArrayToInteger(X25519(B,w)))

ISK = hashlib.sha512(b"ISK").digest()

mac = MAC_SK_AuCPace25519(ISK)

DSI5 = 417543506163653235353139 string (b'AuCPace25519') of len(12)
pw   = 70617373776f7264 (b'password') string of len(8)
ZPAD = 108 zero bytes (before mixing in adversary controlled variable data)
name = 757365726e616d65 (b'username') string of len(8)
u = SHA512(DSI||password||ZPAD||username) as 512 bit little-endian int:
  0xfec497749d249426e4895e05d0bb4f565aa4423f33d19e6b20aa3837eb77d16e << 256
+ 0xb8f5b1b51c1557c088356d7ca09bd78f259f1d5f4041d466f4edd40f041a0bb3
u as reduced base field element coordinate:
  0xa242d046f835586749962599c699e609a00f2c0f15f584dce322c5bf7e327be
Elligator2 output Z as base field element coordinate:
  0x3c01681e4c6d4a43b527c789bcf6046b53ac14c516dfbbbb0f8916826b537f4b




Z for username, password:
0x3c01681e4c6d4a43b527c789bcf6046b53ac14c516dfbbbb0f8916826b537f4b
q:
0xf40546b4544bae5fcc564dc917407ee02300312c8fd558a0b37f48322277962e
R:
0x5698d57693a684a957ae0492bade0dfd6a261dfa2bb4a74c6b0b8b84acf082a8
U:
0xaf2a058fbd974a6122f8316323060d808705701971666121477e

In [130]:
# Test vectors for the C Code

print (ByteArrayToCInitializer(IntegerToByteArray(etc1_in), "EllTestCase1_in"))
print (ByteArrayToCInitializer(IntegerToByteArray(etc1_out), "EllTestCase1_out"))

print (ByteArrayToCInitializer(IntegerToByteArray(etc2_in), "EllTestCase2_in"))
print (ByteArrayToCInitializer(IntegerToByteArray(etc2_out), "EllTestCase2_out"))

tc3_in = bytearray(hashlib.sha512(b"LongInput").digest())
print (ByteArrayToCInitializer(tc3_in, "EllTestCase3_in"))

tc3_out = elligator2_curve25519(tc3_in)
print (ByteArrayToCInitializer(tc3_out, "EllTestCase3_out"))




const uint8_t EllTestCase1_in[] = {
 0xbc,0x14,0x9a,0x46,0xd2,0x93,0xb0,0xae,0xea,0x34,0x58,0x13,0x49,0xd7,0x2f,0x8a,0x5a,0x96,0xcd,0x53,0x11,0x2,0xd6,0x73,0x79,0xcd,0x9b,0xfa,0xdd,0x4e,0xc8,0x0,
};
const uint8_t EllTestCase1_out[] = {
 0x66,0xb6,0x8f,0x75,0x75,0xcd,0x28,0x24,0x3,0xfc,0x2b,0xd3,0x23,0xff,0x4,0x60,0x12,0x3,0xc1,0xec,0x55,0x16,0xce,0x24,0x7f,0x7c,0xa,0xdb,0xef,0x5,0xd3,0x67,
};
const uint8_t EllTestCase2_in[] = {
 0x89,0xcf,0x55,0xd4,0xb5,0xd3,0xf8,0x4b,0x16,0x34,0x95,0x7a,0xc5,0x3,0xa3,0x2b,0x84,0xba,0x11,0x47,0x1a,0x96,0xb2,0x27,0xbc,0xa7,0xa,0xc,0x3b,0xf2,0x63,0x75,
};
const uint8_t EllTestCase2_out[] = {
 0x1d,0xb1,0x63,0xc8,0x6c,0xec,0xa7,0x62,0x19,0x3,0xc9,0x41,0x2d,0x6d,0xc7,0x1b,0x4e,0xd2,0x63,0xb6,0x87,0xee,0xd0,0x92,0xb1,0x94,0xb5,0xe5,0x40,0xbb,0xa3,0x8,
};
const uint8_t EllTestCase3_in[] = {
 0xbf,0x60,0x87,0x60,0x92,0x15,0x2,0xc2,0x52,0xa3,0xf7,0xb2,0xd8,0xb1,0xfb,0x75,0x4c,0x7c,0x96,0xd6,0x7a,0x7b,0x49,0x30,0xf3,0xa8,0xbe,0xa7,0x82,0xd6,0x9e,0x5a,0xfa,0x55,

In [138]:
print (ByteArrayToCInitializer(Z, "tc_Z"))
print (ByteArrayToCInitializer(r, "tc_r"))
print (ByteArrayToCInitializer(U, "tc_U"))
print (ByteArrayToCInitializer(q, "tc_q"))
print (ByteArrayToCInitializer(UQ, "tc_UQ"))
print (ByteArrayToCInitializer(ZQ, "tc_ZQ"))


const uint8_t tc_Z[] = {
 0x4b,0x7f,0x53,0x6b,0x82,0x16,0x89,0xf,0xbb,0xbb,0xdf,0x16,0xc5,0x14,0xac,0x53,0x6b,0x4,0xf6,0xbc,0x89,0xc7,0x27,0xb5,0x43,0x4a,0x6d,0x4c,0x1e,0x68,0x1,0x3c,
};
const uint8_t tc_r[] = {
 0xa8,0x82,0xf0,0xac,0x84,0x8b,0xb,0x6b,0x4c,0xa7,0xb4,0x2b,0xfa,0x1d,0x26,0x6a,0xfd,0xd,0xde,0xba,0x92,0x4,0xae,0x57,0xa9,0x84,0xa6,0x93,0x76,0xd5,0x98,0x56,
};
const uint8_t tc_U[] = {
 0x77,0xa9,0x86,0x73,0xa9,0xeb,0x77,0x14,0x12,0x66,0x16,0x97,0x1,0x57,0x70,0x8,0xd8,0x60,0x30,0x32,0x16,0x83,0x2f,0x12,0xa6,0x74,0xd9,0xfb,0x58,0xa0,0xf2,0xa,
};
const uint8_t tc_q[] = {
 0x28,0x96,0x77,0x22,0x32,0x48,0x7f,0xb3,0xa0,0x58,0xd5,0x8f,0x2c,0x31,0x0,0x23,0xe0,0x7e,0x40,0x17,0xc9,0x4d,0x56,0xcc,0x5f,0xae,0x4b,0x54,0xb4,0x46,0x5,0x74,
};
const uint8_t tc_UQ[] = {
 0xb5,0x6c,0xe,0xe7,0x2b,0x7a,0xa7,0x60,0x55,0xf6,0x95,0x9d,0x64,0x87,0x76,0xfe,0x1b,0xfa,0xf8,0xe0,0x57,0xc0,0xde,0x7a,0x5b,0xb,0x54,0xff,0xda,0x70,0x2,0x61,
};
const uint8_t tc_ZQ[] = {
 0x50,0x9a,0x3a,0x7c,0xf,0xa3,0xc0,0x

In [31]:
print (ByteArrayToCInitializer(G, "tc_G"))
print (ByteArrayToCInitializer(ya, "tc_ya"))
print (ByteArrayToCInitializer(Ya, "tc_Ya"))
print (ByteArrayToCInitializer(yb, "tc_yb"))
print (ByteArrayToCInitializer(Yb, "tc_Yb"))
print (ByteArrayToCInitializer(K1, "tc_K"))
print (ByteArrayToCInitializer(ISK, "tc_ISK"))



const uint8_t tc_G[] = {
 0xc4,0xe,0xe0,0x76,0xcf,0xb7,0xa1,0xfd,0x47,0x54,0x5a,0x6,0x41,0x5a,0x10,0x91,0x89,0xda,0x47,0x5b,0xcb,0x4d,0xd6,0x63,0xf1,0x14,0xd6,0x49,0x71,0xa2,0xbf,0x41,
};
const uint8_t tc_ya[] = {
 0xd0,0x22,0xb7,0x98,0xe3,0xbe,0x24,0x97,0xd5,0x5,0xd1,0x0,0xe4,0x54,0xd,0xe4,0x3f,0xdf,0xa,0xeb,0xa9,0xeb,0xa3,0x75,0x42,0x99,0x44,0x41,0x33,0x93,0xec,0x7f,
};
const uint8_t tc_Ya[] = {
 0xea,0x1a,0x2b,0xc0,0xaa,0xd5,0x38,0x76,0x16,0x1,0xe3,0xa1,0x9e,0xe6,0x94,0x3b,0xf8,0x8b,0xc,0x98,0x1b,0x4b,0xfb,0x6d,0x8c,0x73,0x77,0xf9,0x1c,0xc8,0x4e,0x8,
};
const uint8_t tc_yb[] = {
 0xc0,0xec,0xc,0xd6,0x84,0x32,0x5,0x3c,0xcd,0x6f,0xd4,0xd6,0x4a,0x8,0x20,0x3e,0x8b,0xf2,0xb1,0x3c,0x49,0x58,0x90,0xb5,0x4c,0x87,0xaf,0xfc,0xf3,0x6f,0x6a,0x71,
};
const uint8_t tc_Yb[] = {
 0x5d,0x6e,0x6b,0xd7,0xad,0x8a,0x5c,0xc4,0xfb,0xb0,0xfa,0x8e,0x76,0x63,0x1c,0x91,0x45,0x49,0xe5,0x62,0x75,0x2d,0xf8,0xd2,0x8c,0xd6,0x9,0xa9,0xf,0x35,0xc7,0x29,
};
const uint8_t tc_K[] = {
 0xdc,0x70,0xf,0x2b,0x7e,0xce,0x7a,